In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
train = pd.read_csv("train.csv")
train.head()
train.describe()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,15120.00000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,...,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000
mean,7560.50000,2749.322553,156.676653,16.501587,227.195701,51.076521,1714.023214,212.704299,218.965608,135.091997,...,0.045635,0.040741,0.001455,0.006746,0.000661,0.002249,0.048148,0.043452,0.030357,4.000000
std,4364.91237,417.678187,110.085801,8.453927,210.075296,61.239406,1325.066358,30.561287,22.801966,45.895189,...,0.208699,0.197696,0.038118,0.081859,0.025710,0.047368,0.214086,0.203880,0.171574,2.000066
min,1.00000,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3780.75000,2376.000000,65.000000,10.000000,67.000000,5.000000,764.000000,196.000000,207.000000,106.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,7560.50000,2752.000000,126.000000,15.000000,180.000000,32.000000,1316.000000,220.000000,223.000000,138.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,11340.25000,3104.000000,261.000000,22.000000,330.000000,79.000000,2270.000000,235.000000,235.000000,167.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
max,15120.00000,3849.000000,360.000000,52.000000,1343.000000,554.000000,6890.000000,254.000000,254.000000,248.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [6]:
# train = train.sample(frac=1)
split_val = 11340

x_train = train.drop(["Cover_Type" , "Id"] , axis=1)[:split_val]
y_train_numbers = train["Cover_Type"][:split_val]

x_test = train.drop(["Cover_Type"] , axis=1)[split_val+1:]
y_test_numbers = train["Cover_Type"][split_val+1:]

with tf.Session() as session :
    y_train = tf.one_hot(y_train_numbers , y_train_numbers.max()).eval()
    y_test = tf.one_hot(y_test_numbers , y_train_numbers.max()).eval()
    
x_train.shape , y_train.shape , x_test.shape , y_test.shape

((11340, 54), (11340, 7), (3779, 55), (3779, 7))

In [7]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_train_scaled[0]

array([-0.36903828, -0.95802465, -1.59332183,  0.13765894, -0.83711492,
       -0.90647129,  0.2758905 ,  0.5684083 ,  0.27838072,  4.33266309,
        1.7997923 , -0.18771859, -0.85222658, -0.66925782, -0.15528267,
       -0.20862951, -0.26172676, -0.23925735, -0.10428508, -0.21544155,
        0.        , -0.00939102, -0.02300827, -0.40338463, -0.16512711,
       -0.12555859, -0.18210784, -0.10893844,  0.        , -0.08428995,
       -0.20257228, -0.06381974, -0.05564149, -0.09759869, -0.03387772,
       -0.14952744, -0.23343383, -0.13364861, -0.00939102, -0.06023819,
       -0.03254723, -0.02100266,  3.29198432, -0.22208081, -0.1501427 ,
       -0.22229933, -0.20816881, -0.03639373, -0.08159528, -0.02657001,
       -0.04605313, -0.22490811, -0.21499297, -0.17339639])

In [207]:
input_size = 55
hidden1_size = 50
hidden2_size = 25
output_size = 7

weights = {
    "h1": tf.Variable(tf.random_normal([input_size , hidden1_size])) ,
    "h2": tf.Variable(tf.random_normal([hidden1_size , hidden2_size])) ,
    "out": tf.Variable(tf.random_normal([hidden2_size , output_size]))
}
bias = {
    "h1": tf.Variable(tf.random_normal([hidden1_size])) ,
    "h2": tf.Variable(tf.random_normal([hidden2_size])) ,
    "out": tf.Variable(tf.random_normal([output_size]))
}

In [208]:
def fwd_prop(x , weights , bias):
    in_l1 = tf.add(tf.matmul(x , weights["h1"]) , bias["h1"])
    out_l1 = tf.nn.relu(in_l1)
    
    in_l2 = tf.add(tf.matmul(out_l1 , weights["h2"]) , bias["h2"])
    out_l2 = tf.nn.relu(in_l2)
    
    output = tf.add(tf.matmul(out_l2 , weights["out"]) , bias["out"])
    
    return output

In [209]:
x_tensor = tf.placeholder("float" , [None , input_size])
y_tensor = tf.placeholder(tf.int32 , [None , output_size])
y_tensor_pred = fwd_prop(x_tensor , weights , bias)
cost_tensor = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_tensor_pred , labels=y_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost_tensor)

In [210]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [216]:
batch_size = 50
for i in range(50) :
    num_batches = int(np.ceil(x_train.shape[0] / batch_size))
    for j in range(num_batches):
        batch_x = x_train[batch_size*j : batch_size*(j+1)]
        batch_y = y_train[batch_size*j : batch_size*(j+1)]
        if (batch_size*(j+1)) > x_train.shape[0] :
            batch_x = x_train[batch_size*j : ]
            batch_y = y_train[batch_size*j : ]
        
        min_cost , opt_ = session.run([cost_tensor , optimize] , feed_dict={x_tensor: batch_x , y_tensor: batch_y})

In [217]:
y_pred = tf.argmax(y_tensor_pred , axis=1)
y_true = tf.argmax(y_train , axis=1)
pred_outputs , real_outputs = session.run([y_pred , y_true] , feed_dict={x_tensor: x_train_scaled , y_tensor: y_train})
score_tensor = tf.equal(pred_outputs , real_outputs)
score = session.run(score_tensor)
score.sum()/x_train.shape[0]

0.15564373897707232

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split , GridSearchCV

In [9]:
x_train , x_test , y_train , y_test = train_test_split(train.drop(["Cover_Type" , "Id"] , axis=1) , train["Cover_Type"])
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)

In [251]:
clf = RandomForestClassifier(n_jobs=-1 , max_features="sqrt")
grid_params = {"max_depth": [i for i in range(50 , 56)] , "min_samples_split":[i for i in range(3 , 10)]}
grid = GridSearchCV(clf , grid_params , verbose=2)
grid.fit(x_train_scaled , y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV] max_depth=50, min_samples_split=3 ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ max_depth=50, min_samples_split=3, total=   0.9s
[CV] max_depth=50, min_samples_split=3 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] ................ max_depth=50, min_samples_split=3, total=   0.9s
[CV] max_depth=50, min_samples_split=3 ...............................
[CV] ................ max_depth=50, min_samples_split=3, total=   0.9s
[CV] max_depth=50, min_samples_split=3 ...............................
[CV] ................ max_depth=50, min_samples_split=3, total=   0.8s
[CV] max_depth=50, min_samples_split=3 ...............................
[CV] ................ max_depth=50, min_samples_split=3, total=   0.9s
[CV] max_depth=50, min_samples_split=4 ...............................
[CV] ................ max_depth=50, min_samples_split=4, total=   0.8s
[CV] max_depth=50, min_samples_split=4 ...............................
[CV] ................ max_depth=50, min_samples_split=4, total=   0.9s
[CV] max_depth=50, min_samples_split=4 ...............................
[CV] ................ max_depth=50, min_samples_split=4, total=   1.1s
[CV] max_depth=50, min_samples_split=4 ...............................
[CV] .

[CV] ................ max_depth=51, min_samples_split=7, total=   1.2s
[CV] max_depth=51, min_samples_split=8 ...............................
[CV] ................ max_depth=51, min_samples_split=8, total=   1.0s
[CV] max_depth=51, min_samples_split=8 ...............................
[CV] ................ max_depth=51, min_samples_split=8, total=   1.2s
[CV] max_depth=51, min_samples_split=8 ...............................
[CV] ................ max_depth=51, min_samples_split=8, total=   1.1s
[CV] max_depth=51, min_samples_split=8 ...............................
[CV] ................ max_depth=51, min_samples_split=8, total=   1.2s
[CV] max_depth=51, min_samples_split=8 ...............................
[CV] ................ max_depth=51, min_samples_split=8, total=   1.1s
[CV] max_depth=51, min_samples_split=9 ...............................
[CV] ................ max_depth=51, min_samples_split=9, total=   1.2s
[CV] max_depth=51, min_samples_split=9 ...............................
[CV] .

[CV] ................ max_depth=53, min_samples_split=5, total=   1.2s
[CV] max_depth=53, min_samples_split=5 ...............................
[CV] ................ max_depth=53, min_samples_split=5, total=   1.2s
[CV] max_depth=53, min_samples_split=5 ...............................
[CV] ................ max_depth=53, min_samples_split=5, total=   1.3s
[CV] max_depth=53, min_samples_split=6 ...............................
[CV] ................ max_depth=53, min_samples_split=6, total=   1.0s
[CV] max_depth=53, min_samples_split=6 ...............................
[CV] ................ max_depth=53, min_samples_split=6, total=   1.1s
[CV] max_depth=53, min_samples_split=6 ...............................
[CV] ................ max_depth=53, min_samples_split=6, total=   1.1s
[CV] max_depth=53, min_samples_split=6 ...............................
[CV] ................ max_depth=53, min_samples_split=6, total=   1.2s
[CV] max_depth=53, min_samples_split=6 ...............................
[CV] .

[CV] ................ max_depth=55, min_samples_split=3, total=   1.2s
[CV] max_depth=55, min_samples_split=3 ...............................
[CV] ................ max_depth=55, min_samples_split=3, total=   1.1s
[CV] max_depth=55, min_samples_split=3 ...............................
[CV] ................ max_depth=55, min_samples_split=3, total=   1.3s
[CV] max_depth=55, min_samples_split=3 ...............................
[CV] ................ max_depth=55, min_samples_split=3, total=   1.2s
[CV] max_depth=55, min_samples_split=3 ...............................
[CV] ................ max_depth=55, min_samples_split=3, total=   1.1s
[CV] max_depth=55, min_samples_split=4 ...............................
[CV] ................ max_depth=55, min_samples_split=4, total=   1.3s
[CV] max_depth=55, min_samples_split=4 ...............................
[CV] ................ max_depth=55, min_samples_split=4, total=   1.3s
[CV] max_depth=55, min_samples_split=4 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done 210 out of 210 | elapsed:  4.0min finished


GridSearchCV(estimator=RandomForestClassifier(max_features='sqrt', n_jobs=-1),
             param_grid={'max_depth': [50, 51, 52, 53, 54, 55],
                         'min_samples_split': [3, 4, 5, 6, 7, 8, 9]},
             verbose=2)

In [253]:
grid.best_estimator_ , grid.best_score_

(RandomForestClassifier(max_depth=55, max_features='sqrt', min_samples_split=4,
                        n_jobs=-1),
 0.8495590828924161)

In [10]:
clf = RandomForestClassifier(max_depth=55, max_features='sqrt', min_samples_split=4, n_jobs=-1)
clf.fit(x_train_scaled , y_train)

x_test_scaled = scaler.transform(x_test)
clf.score(x_test_scaled , y_test)

0.8507936507936508

In [11]:
x = train.drop(["Cover_Type" , "Id"] , axis=1)
y = train["Cover_Type"]
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

In [256]:
clf = RandomForestClassifier(n_jobs=-1 , max_features="sqrt")
grid_params = {"max_depth": [51 , 52 , 53 , 54 , 55] , "min_samples_split":[3 , 4 , 5 , 6]}
grid = GridSearchCV(clf , grid_params , verbose=2)
grid.fit(x_scaled , y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=51, min_samples_split=3 ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ max_depth=51, min_samples_split=3, total=   1.0s
[CV] max_depth=51, min_samples_split=3 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] ................ max_depth=51, min_samples_split=3, total=   1.0s
[CV] max_depth=51, min_samples_split=3 ...............................
[CV] ................ max_depth=51, min_samples_split=3, total=   1.0s
[CV] max_depth=51, min_samples_split=3 ...............................
[CV] ................ max_depth=51, min_samples_split=3, total=   1.0s
[CV] max_depth=51, min_samples_split=3 ...............................
[CV] ................ max_depth=51, min_samples_split=3, total=   1.8s
[CV] max_depth=51, min_samples_split=4 ...............................
[CV] ................ max_depth=51, min_samples_split=4, total=   1.8s
[CV] max_depth=51, min_samples_split=4 ...............................
[CV] ................ max_depth=51, min_samples_split=4, total=   1.5s
[CV] max_depth=51, min_samples_split=4 ...............................
[CV] ................ max_depth=51, min_samples_split=4, total=   1.4s
[CV] max_depth=51, min_samples_split=4 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.1min finished


GridSearchCV(estimator=RandomForestClassifier(max_features='sqrt', n_jobs=-1),
             param_grid={'max_depth': [51, 52, 53],
                         'min_samples_split': [3, 4, 5]},
             verbose=2)

In [12]:
grid.best_estimator_ , grid.best_score_

NameError: name 'grid' is not defined

In [14]:
clf = RandomForestClassifier(max_depth=51, max_features='sqrt', min_samples_split=4, n_jobs=-1)
clf.fit(x_scaled , y)

RandomForestClassifier(max_depth=51, max_features='sqrt', min_samples_split=4,
                       n_jobs=-1)

In [15]:
x_test = pd.read_csv("test.csv").drop("Id" , axis=1)
x_test.head()
x_test_scaled = scaler.transform(x_test)

In [16]:
soln = pd.DataFrame()
soln["Id"] = pd.read_csv("test.csv")["Id"]
soln["Cover_Type"] = clf.predict(x_test_scaled)

In [17]:
soln.to_csv("soln.csv" , index=False)